# Everything I Need To Know About ETL, I Learned The Hard Way

@catherinedevlin

Jan 8, 2018

# Everything I Need To Know About ETL, I Hope To Learn Eventually

# ETL

## Extract

## Transform 
 
## Load

# DATA Act

- TTS / Treasury / Booz Allen Hamilton
- https://github.com/fedspendingtransparency
- @kaitlin and @becky
- [explanatory post](https://18f.gsa.gov/2015/06/09/data-act-data-act-explainer/) and [tag](https://18f.gsa.gov/tags/data-act/)

![data act data flow schematic](images/data_act_flow.jpg)

- Often described as between technologies (CSV/XML/etc. to RDBMS)

- Reshaping the data usually harder 

# 1. Respect quantity

- Out of band so "no problem"?
- Only need to do it "once"?

# 1a. Do the math

1000 rows / 30 seconds

In [7]:
(10000000. / (1000. / 30.) / 3600.)

83.33333333333333

- Connections breaking
- Colliding do-overs


# 1b. Consider set-based

    for row in rows:
        row.name = row.name.strip().upper()
    
vs

    UPDATE my_table
    SET    name = UPPER(TRIM(name));


# Set-based options

- Raw SQL
- Pandas
- Django's `.bulk_create`, `.update` (warning: no `.save`)

Aim for consistency

# 2. data model modesty

# "Data model should fit API should fit website"

![schematic of original ETL plan](images/original_plan.jpg)

# Flaw in plan

Initial design for website != final design

![schematic of unfortunate reality](images/api_masking.jpg)

![schematic of unfortunate ETL flow](images/alas.jpg)

Could have had...

![masked with views](images/masking_views.jpg)

Data models and ETL processes are hard to change

# 3. Understand mandate

    city     county         state
    -----------------------------
    Dayton   Montgomery     OH
    Dayton                  OH
    Dayton   Montgomery  
    Dayton   Mongomery OH   
    
It hurts to see this.  It's so satisfying to fix this.  It improves the quality of the data.

# 4. Leave breadcrumbs

grants

    id    FAIN     recip_city  recip_state   recip_addr
    ------------------------------------------------------
    101   ABC123   Dayton      OH            
    102   ABC456   Dayton      OH            123 Main
    103   XYZ123   Chicago     IL
    104   XYZ789   Dayton      OH   
    
locations

    id    city    state   addr    
    ------------------------------
    1     Dayton  OH      
    2     Dayton  OH      123 Main

Hey, row, 

Which source row(s) do you come from?

- We need to rerun you (idempotency)
- We need to confirm you 
- We need to debug you

Slow in human effort, slow in computation

    id  source_tbl  source_ids   script   commit_id
    -----------------------------------------------
    1   grants      {101, 104}   etl.py   dd87ad
    2   grants      {102}        etl.py   0714a8
    
(in dest table, or associated table)

    INSERT INTO locations
    SELECT recip_city AS city,
           recip_state AS state,
           recip_addr AS addr,
           ARRAY_AGG(id) AS source_ids
    FROM   grants
    GROUP BY 1, 2, 3
    
Also useful: [Common Table Expressions (CTEs) with RETURNING clause](https://rob.conery.io/2015/02/08/inserting-using-new-record-postgres/)

https://github.com/18f/tech-talks